In [1]:
from gol_apis import RosterActivitiesService
from gol_apis import RosterTagService
from gol_apis import CheckScheduleService
from gol_apis import CrewMemberInfoService
import api_tools
from api_tools.setup import setup_crew_info_dataframe
from api_tools.setup import setup_checks_dataframe
from api_tools.setup import setup_tag_dataframe
from api_tools.setup import setup_activities_dataframe

from gol_logger import Logger
import pandas as pd
import multiprocessing as mp
import requests, time
from datetime import datetime as dt
from datetime import timedelta as td
from utils import funcs
from utils.progress import step
from utils.progress import BAR
from itertools import chain

from modules.generate_data import get_data_neolude
from modules.generate_data import get_rosters
from modules.generate_data import get_crew_info
from modules.generate_data import get_checks

import sys
from config.variables import user

In [2]:
training_code = 'SEC'

In [3]:
# Dados Crew Info
crew = pd.read_csv('data/crew/crew_info.csv', sep=';', encoding='utf-8-sig')
crew.cif = crew.cif.astype(int).astype(str).str.zfill(8)
crew = crew[crew['funcao'].isin(['CMT', 'COP'])]
crew = crew[['cif', 'guerra', 'funcao', 'cargo', 'base']]
crew

,cif,guerra,funcao,cargo,base
0,00000020,BORIO,CMT,IBX,CGH
1,00000021,GRABLER,CMT,IBX,GRU
2,00000022,DANTAS,CMT,IBX,CGH
3,00000024,BORBA,CMT,NaN,GRU
4,00000026,FERNANDO FARIAS,CMT,IBX,CGH
...,...,...,...,...,...
5479,99999913,PIAZZA,CMT,IB,GRU
5480,99999916,GALENO CABRAL,CMT,IB,GRU
5481,99999917,EDWARD,CMT,IB,GRU
5482,99999918,CESIDIO,CMT,IB,GRU


In [4]:
# Header para filtrar dados do Neolude
header_neolude = ['RegistrationNumber',
                  'UserName',
                  'BusinessUnitName',
                  'Post',
                  'CPF',
                  'Email',
                  'CourseName',
                  'CourseUserStatus',
                  'GradeFormatted',
                  'LastAccessDate',
                  'CreatedDate',
                  'CompletionDate']

In [5]:
# Dados Neolude 
training = pd.read_csv(f'data/neolude/{training_code}_inicial.csv', sep=';', encoding='utf-8-sig')
training = training[header_neolude]
training['RegistrationNumber'] = training['RegistrationNumber'].fillna(0)
training['RegistrationNumber'] = training['RegistrationNumber'].astype(int).astype(str).str.zfill(8)
training['LastAccessDate'] = pd.to_datetime(training['LastAccessDate'])
training['CreatedDate'] = pd.to_datetime(training['CreatedDate'])
training['CompletionDate'] = pd.to_datetime(training['CompletionDate'])
training.columns = ['cif', 'name', 'base', 'funcao', 'cpf', 'email', 'course_name','status', 'nota', 'ultimo_acesso', 'criacao_acesso', 'data_conclusao']
training = training[['cif', 'course_name','status', 'nota', 'data_conclusao']]
training = training.sort_values(by=['cif','data_conclusao'])
training = training.drop_duplicates(subset = ['cif'], keep = 'last')
training

,cif,course_name,status,nota,data_conclusao
995,00000000,AVSEC PARA TRIPULANTES - FORMAÇÃO RBAC 110,Concluído,"10,00",2021-01-09 17:30:00
1746,00000020,AVSEC PARA TRIPULANTES - FORMAÇÃO RBAC 110,Concluído,"10,00",2020-11-18 14:27:31
1220,00000026,AVSEC PARA TRIPULANTES - FORMAÇÃO RBAC 110,Concluído,"0,00",2018-11-23 00:00:00
1506,00000126,AVSEC PARA TRIPULANTES - FORMAÇÃO RBAC 110,Concluído,"10,00",2018-10-05 00:00:00
850,00000156,AVSEC PARA TRIPULANTES - FORMAÇÃO RBAC 110,Concluído,"10,00",2019-09-30 00:00:00
...,...,...,...,...,...
658,00045994,AVSEC PARA TRIPULANTES - FORMAÇÃO RBAC 110,Concluído,"9,00",2020-03-09 00:00:00
517,00045997,AVSEC PARA TRIPULANTES - FORMAÇÃO RBAC 110,Concluído,"9,00",2020-03-09 00:00:00
1749,00046021,AVSEC PARA TRIPULANTES - FORMAÇÃO RBAC 110,Concluído,"10,00",2020-03-09 00:00:00
1827,00436588,AVSEC PARA TRIPULANTES - FORMAÇÃO RBAC 110,Concluído,"10,00",2021-01-09 17:30:00


In [6]:
# Dados de Rosters
rosters = funcs.generate_data('data/rosters/')
rosters.cif = rosters.cif.astype(str).str.zfill(8)

importing: 2020-01...
importing: 2020-02...
importing: 2020-03...
importing: 2020-04...
importing: 2020-05...
importing: 2020-06...
importing: 2020-07...
importing: 2020-08...
importing: 2020-09...
importing: 2020-10...
importing: 2020-11...
importing: 2020-12...
importing: 2021-01...


In [7]:
inss = rosters.copy()
inss['dep'] = pd.to_datetime(inss['dep'], format='%Y-%m-%d')
inss['dep'] = inss['dep'].dt.floor('D')
inss = inss[(inss['code'] == 'INSS') & (inss['dep'] == pd.to_datetime(dt.date(dt.now())))]
inss = inss[['cif', 'code']]
inss.columns = ['cif', 'inss']
inss

,cif,inss
65631,00001962,INSS
65632,00002329,INSS
65634,00002976,INSS
65643,00010705,INSS
65648,00004196,INSS
...,...,...
66957,00005916,INSS
66959,00006334,INSS
66976,00041650,INSS
66978,00042249,INSS


In [8]:
lnr = rosters.copy()
lnr['dep'] = pd.to_datetime(lnr['dep'], format='%Y-%m-%d')
lnr['dep'] = lnr['dep'].dt.floor('D')
lnr = lnr[(lnr['code'] == 'LIC-SREM') & (lnr['dep'] == pd.to_datetime(dt.date(dt.now())))]
lnr = lnr[['cif', 'code']]
lnr.columns = ['cif', 'lnr']
lnr

,cif,lnr
65635,00023872,LIC-SREM
65636,00023903,LIC-SREM
65638,00024141,LIC-SREM
65649,00004453,LIC-SREM
65651,00004615,LIC-SREM
...,...,...
66972,00046284,LIC-SREM
66973,00046328,LIC-SREM
66974,00046330,LIC-SREM
66975,00046333,LIC-SREM


In [9]:
rosters = rosters[rosters['code'].isin(funcs.training_filter(training_code))]
rosters = rosters.sort_values(by=['cif','dep'])
rosters = rosters[['cif', 'code', 'fr', 'dep', 'arr']]
rosters['duration'] = pd.to_datetime(rosters['arr']) - pd.to_datetime(rosters['dep'])
rosters['duration'] = rosters['duration'].astype('timedelta64[h]').astype(int)
rosters = rosters[rosters['duration'] > 6.0]
rosters = rosters[['cif', 'code', 'fr', 'dep']]
rosters = rosters.drop_duplicates(subset = ['cif'], keep = 'last')
rosters

,cif,code,fr,dep
105258,00000020,C-ENS-SEC,CGH,2020-11-18 11:30:00
20514,00000238,C-ENS-SEC,CGH,2020-12-04 11:30:00
105262,00000331,C-ENS-SEC,CGH,2020-11-18 11:30:00
25700,00000332,C-ENS-SEC,CGH,2020-11-05 11:30:00
25667,00000461,C-ENS-SEC,CGH,2020-11-05 11:30:00
...,...,...,...,...
61292,00045989,C-ENS-SEC,CGH,2020-03-09 11:30:00
61279,00045992,C-ENS-SEC,CGH,2020-03-09 11:30:00
61338,00045994,C-ENS-SEC,CGH,2020-03-09 11:30:00
61315,00045997,C-ENS-SEC,CGH,2020-03-09 11:30:00


In [10]:
# Dados de Checks
checks = pd.read_csv('data/checks/check_schedule.csv', sep=';', encoding='utf-8-sig')
checks.cif = checks.cif.astype(str).str.zfill(8)
checks = checks[checks['checkTypeCode'] == training_code]
checks = checks[['cif', 'checkTypeCode', 'expiryDt']]
checks

,cif,checkTypeCode,expiryDt
63,00046225,SEC,2022-09-30
73,00034571,SEC,2022-10-31
74,00002976,SEC,2020-11-27
78,00024543,SEC,2022-12-31
81,00024238,SEC,2022-12-31
...,...,...,...
54868,00005686,SEC,2021-11-30
54869,00022337,SEC,2021-11-30
54980,00038070,SEC,2022-01-31
54981,00042289,SEC,2022-01-31


In [11]:
# Merge dos dados
data_all = pd.merge(crew, checks[['cif', 'checkTypeCode', 'expiryDt']], on='cif', how='left')
data_all = data_all.merge(training, on='cif', how='left')
data_all = data_all.merge(rosters[['cif', 'code', 'dep']], on='cif', how='left')
data_all = data_all.merge(inss, on='cif', how='left')
data_all = data_all.merge(lnr, on='cif', how='left')
data_all

,cif,guerra,funcao,cargo,base,checkTypeCode,expiryDt,course_name,status,nota,data_conclusao,code,dep,inss,lnr
0,00000020,BORIO,CMT,IBX,CGH,SEC,2022-11-30,AVSEC PARA TRIPULANTES - FORMAÇÃO RBAC 110,Concluído,"10,00",2020-11-18 14:27:31,C-ENS-SEC,2020-11-18 11:30:00,NaN,NaN
1,00000021,GRABLER,CMT,IBX,GRU,SEC,2021-01-28,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN
2,00000022,DANTAS,CMT,IBX,CGH,SEC,2022-09-30,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN
3,00000024,BORBA,CMT,NaN,GRU,SEC,2022-09-30,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN
4,00000026,FERNANDO FARIAS,CMT,IBX,CGH,SEC,2022-11-30,AVSEC PARA TRIPULANTES - FORMAÇÃO RBAC 110,Concluído,"0,00",2018-11-23 00:00:00,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2029,99999913,PIAZZA,CMT,IB,GRU,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN
2030,99999916,GALENO CABRAL,CMT,IB,GRU,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN
2031,99999917,EDWARD,CMT,IB,GRU,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN
2032,99999918,CESIDIO,CMT,IB,GRU,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN


In [12]:
# Formatação do dataFrame de Dados
validation = data_all.copy()

# Ajuste de data do vencimento
validation['expiryDt'] = pd.to_datetime(validation['expiryDt'], format='%Y-%m-%d')
validation['expiryDt'] = validation['expiryDt'].dt.floor('D')

validation['dep'] = validation['dep'].fillna(pd.to_datetime(dt(2035,12,31)))
validation['dep'] = pd.to_datetime(validation['dep'], format='%Y-%m-%d')
validation['dep'] = validation['dep'].dt.floor('D')

validation['data_conclusao'] = validation['data_conclusao'].fillna(pd.to_datetime(dt(2035,12,31)))
validation['data_conclusao'] = pd.to_datetime(validation['data_conclusao'], format='%Y-%m-%d')
validation['data_conclusao'] = validation['data_conclusao'].dt.floor('D')

validation = validation[validation['expiryDt'] < (pd.to_datetime(dt.date(dt.now())) + pd.offsets.MonthBegin(0))]
validation['tempo_de_conclusao (dias)'] = pd.to_datetime(dt.date(dt.now())) - validation['data_conclusao']

validation['tempo_de_conclusao (dias)'] = validation['tempo_de_conclusao (dias)'].dt.days

validation = validation[validation['dep'] < (pd.to_datetime(dt.date(dt.now())))]
validation['atualizar'] = (validation['status'] == 'Concluído') & (validation['tempo_de_conclusao (dias)'] < 90)

validation

,cif,guerra,funcao,cargo,base,checkTypeCode,expiryDt,course_name,status,nota,data_conclusao,code,dep,inss,lnr,tempo_de_conclusao (dias),atualizar
342,00003090,IVAN,CMT,NaN,CGH,SEC,2016-10-31,AVSEC PARA TRIPULANTES - FORMAÇÃO RBAC 110,Concluído,"10,00",2021-01-07,C-ENS-SEC,2021-01-07,NaN,NaN,4,True
805,00009811,FELIPE GRILLO,CMT,IR,GIG,SEC,2020-12-29,AVSEC PARA TRIPULANTES - FORMAÇÃO RBAC 110,Concluído,"10,00",2021-01-07,C-ENS-SEC,2021-01-07,NaN,NaN,4,True
835,00010630,LUIZ SCORSE,CMT,NaN,GIG,SEC,2020-11-30,AVSEC PARA TRIPULANTES - FORMAÇÃO RBAC 110,Concluído,"10,00",2021-01-07,C-ENS-SEC,2021-01-07,NaN,NaN,4,True
876,00011132,FRANCISCO BORGES,CMT,NaN,SDU,SEC,2020-10-28,AVSEC PARA TRIPULANTES - FORMAÇÃO RBAC 110,Concluído,"10,00",2021-01-07,C-ENS-SEC,2021-01-07,NaN,NaN,4,True
910,00011596,BRENNER,CMT,NaN,BSB,SEC,2020-12-29,AVSEC PARA TRIPULANTES - FORMAÇÃO RBAC 110,Concluído,"10,00",2021-01-07,C-ENS-SEC,2021-01-07,NaN,NaN,4,True
1145,00023632,TATIANA MONICO,COP,NaN,GIG,SEC,2020-12-29,AVSEC PARA TRIPULANTES - FORMAÇÃO RBAC 110,Concluído,"10,00",2021-01-07,C-ENS-SEC,2021-01-07,NaN,NaN,4,True


In [13]:
validation.to_csv(f'./data/output/{training_code}_inicial.csv', index=False, sep=';', encoding='utf-8-sig')